# **Summarized results**

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

import visualization

In [ ]:
MEAS_ROOT = "../02_data/"
MEAS_CASES = ["mix10", "mix25", "mix50"]
SEEDS = [42, 1812, 9698, 424, 820, 75, 98, 65535, 16383, 513]
PENS = [2, 4, 6, 8, 10]

In [ ]:
def plot_value_with_range(pos_x, data, ax, color, marker, label):
    ax.scatter([pos_x], [np.mean(data)], color=color, marker=marker, label=label)
    ax.vlines(pos_x, np.quantile(data.dropna(), 0.1),
              np.quantile(data.dropna(), 0.9), color=color)

In [ ]:
def read_vehicle_data(meas_series, meas_name, seeds, col_names, col_values):
    meas_path = f"{MEAS_ROOT}/{meas_series}/{meas_name}"
    veh_data_comb = pd.DataFrame()
    for seed in seeds:
        with open(f"{meas_path}/{seed}/veh_results.json") as f:
            veh_res = json.load(f)
        veh_data = pd.DataFrame.from_dict(veh_res, orient="index")
        veh_data["seed"] = seed
        veh_data.index = veh_data.index.astype(int)

        trip_df = pd.read_xml(f"{meas_path}/{seed}/vehicle_trips.xml", xpath = "tripinfo")
        trip_df["id"] = trip_df["id"].astype(int)

        vd = pd.merge(left=trip_df, right=veh_data, left_on="id", right_index=True)
        
        for col_name, col_value in zip(col_names, col_values):
            vd[col_name] = [col_value]*len(vd)
        
        veh_data_comb = pd.concat([veh_data_comb, vd])
    
    #veh_data_comb["prolonged_distance"] = np.clip(veh_data_comb["routeLength"] - veh_data_comb["original_distance"], a_min = 0, a_max=None)
    return veh_data_comb

## Traffic flow

In [ ]:
def read_flow(meas_series, meas_name, seeds):
    flow_series = []
    for seed in SEEDS:
        flow_df = pd.read_xml(f"{MEAS_ROOT}/{meas_series}/{meas_name}/{seed}/detector_data.out.xml", xpath="interval")
        flow_series.append(flow_df.groupby("begin")["flow"].mean().values[4:14])
    flows = np.array(flow_series)
    return flows

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 3))

ax.set_xticks([0, 3.5, 6.5, 9.5, 12.5, 15.5,
              21, 24.5, 27.5, 30.5, 33.5, 36.5,
              42, 45.5, 48.5, 51.5, 54.5, 57.5],
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]
             )

sec = ax.secondary_xaxis(location=-0.1)
sec.set_xticks([7, 7+21, 7+21*2], MEAS_CASES)
ax.fill_between([-3, 18], 0, 170, color="gray", alpha=0.1, zorder=0)
ax.fill_between([18, 39], 0, 170, color="tab:blue", alpha=0.1, zorder=0)
ax.fill_between([39, 60], 0, 170, color="gray", alpha=0.1, zorder=0)

for i, mc in enumerate(MEAS_CASES):
    flow = read_flow(mc, "baseline", SEEDS)
    plot_value_with_range(21*i, pd.DataFrame(flow), ax, "k", "o",
                          "baseline" if i==0 else None)
    print(f"baseline, {mc}: {np.mean(flow)}")
    
    for j, pen in enumerate(PENS):
        flow = read_flow(mc, f"information{pen}", SEEDS)
        plot_value_with_range(21*i+j*3+3, pd.DataFrame(flow), ax, "tab:blue", "v",
                      "information" if i==0 and j==0 else None)
        print(f"info, {mc}, {pen}: {np.mean(flow)}")
        flow = read_flow(mc, f"auction{pen}", SEEDS)
        plot_value_with_range(21*i+j*3+4, pd.DataFrame(flow), ax, "navy", "^",
                      "auction" if i==0 and j==0 else None)
        print(f"auction, {mc}, {pen}: {np.mean(flow)}")
ax.set_ylim(130, 160)
ax.set_xlim(-3, 60)
ax.set_ylabel("avg. vehicle flow [veh/h]")
ax.legend()

## #reroutes

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 3))

ax.set_xticks([0, 3.5, 6.5, 9.5, 12.5, 15.5,
              21, 24.5, 27.5, 30.5, 33.5, 36.5,
              42, 45.5, 48.5, 51.5, 54.5, 57.5],
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]
             )

sec = ax.secondary_xaxis(location=-0.15)
sec.set_xticks([7, 7+21, 7+21*2], MEAS_CASES)
ax.fill_between([-3, 18], -0.5, 5.5, color="gray", alpha=0.1, zorder=0)
ax.fill_between([18, 39], -0.5, 5.5, color="tab:blue", alpha=0.1, zorder=0)
ax.fill_between([39, 60], -0.5, 5.5, color="gray", alpha=0.1, zorder=0)

for i, mc in enumerate(MEAS_CASES):
    vd = read_vehicle_data(mc, "baseline", SEEDS, [], [])
    plot_value_with_range(21*i, vd["rerouteNo"], ax, "k", "o",
                          "baseline" if i==0 else None)
    print(f"baseline, {mc}: {np.mean(vd['rerouteNo'])}")
    
    for j, pen in enumerate(PENS):
        vd = read_vehicle_data(mc, f"information{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+3, vd["rerouteNo"], ax, "tab:blue", "v",
                      "information" if i==0 and j==0 else None)
        print(f"info, {mc}, {pen}: {np.mean(vd['rerouteNo'])}")
        vd = read_vehicle_data(mc, f"auction{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+4, vd["rerouteNo"], ax, "navy", "^",
                      "auction" if i==0 and j==0 else None)
        print(f"auction, {mc}, {pen}: {np.mean(vd['rerouteNo'])}")
ax.set_ylim(-0.1, 2.2)
ax.set_xlim(-3, 60)
ax.set_ylabel("#reroutes")
ax.legend(loc="upper left")

## Parking distance

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 3))

ax.set_xticks([0, 3.5, 6.5, 9.5, 12.5, 15.5,
              21, 24.5, 27.5, 30.5, 33.5, 36.5,
              42, 45.5, 48.5, 51.5, 54.5, 57.5],
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]
             )

sec = ax.secondary_xaxis(location=-0.15)
sec.set_xticks([7, 7+21, 7+21*2], MEAS_CASES)
ax.fill_between([-3, 18], -20.5, 500, color="gray", alpha=0.1, zorder=0)
ax.fill_between([18, 39], -20.5, 500, color="tab:blue", alpha=0.1, zorder=0)
ax.fill_between([39, 60], -20.5, 500, color="gray", alpha=0.1, zorder=0)

for i, mc in enumerate(MEAS_CASES):
    vd = read_vehicle_data(mc, "baseline", SEEDS, [], [])
    plot_value_with_range(21*i, vd["parking_distance"], ax, "k", "o",
                          f"baseline" if i==0 else None)
    #print(f"baseline, {mc}: {np.mean(vd['parking_distance'])}")
    
    for j, pen in enumerate(PENS):
        vd = read_vehicle_data(mc, f"information{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+3, vd["parking_distance"], ax, "tab:blue", "v",
                      f"information" if i==0 and j==0 else None)
        #print(f"info, {mc}, {pen}: {np.mean(vd['parking_distance'])}")
        vd = read_vehicle_data(mc, f"auction{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+4, vd["parking_distance"], ax, "navy", "^",
                      f"auction" if i==0 and j==0 else None)
        #print(f"auction, {mc}, {pen}: {np.mean(vd['parking_distance'])}")
ax.set_ylim(-10, 450)
ax.set_xlim(-3, 60)
ax.set_ylabel("parking distance [m]")
ax.legend(loc="upper right")

## Paid prices

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 3))

ax.set_xticks([0, 3.5, 6.5, 9.5, 12.5, 15.5,
              21, 24.5, 27.5, 30.5, 33.5, 36.5,
              42, 45.5, 48.5, 51.5, 54.5, 57.5],
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]+
              ["0%"] + [f"{10*i}%" for i in range(2, 12, 2)]
             )

sec = ax.secondary_xaxis(location=-0.15)
sec.set_xticks([7, 7+21, 7+21*2], MEAS_CASES)
ax.fill_between([-3, 18], -20.5, 500, color="gray", alpha=0.1, zorder=0)
ax.fill_between([18, 39], -20.5, 500, color="tab:blue", alpha=0.1, zorder=0)
ax.fill_between([39, 60], -20.5, 500, color="gray", alpha=0.1, zorder=0)

for i, mc in enumerate(MEAS_CASES):
    vd = read_vehicle_data(mc, "baseline", SEEDS, [], [])
    plot_value_with_range(21*i, vd["paid_price"], ax, "k", "o",
                          f"baseline" if i==0 else None)
    print(f"baseline, {mc}: {np.mean(vd['paid_price'])}")
    
    for j, pen in enumerate(PENS):
        vd = read_vehicle_data(mc, f"information{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+3, vd["parking_price"], ax, "tab:blue", "v",
                      f"information" if i==0 and j==0 else None)
        print(f"info, {mc}, {pen}: {np.mean(vd['parking_price'])}")
        vd = read_vehicle_data(mc, f"auction{pen}", SEEDS, [], [])
        plot_value_with_range(21*i+j*3+4, vd["paid_price"], ax, "navy", "^",
                      f"auction" if i==0 and j==0 else None)
        print(f"auction, {mc}, {pen}: {np.mean(vd['paid_price'])}")
ax.set_ylim(0.45, 1.15)
ax.set_xlim(-3, 60)
ax.set_ylabel("actual paid price [€]")
ax.legend(loc="upper left", ncols=3)

## Auction-related data

In [ ]:
vehicle_data = pd.DataFrame()
for meas_case in MEAS_CASES:
    for pen in [0]+PENS:
        vd_ = read_vehicle_data(meas_case, f"auction{pen}", SEEDS,
                                ["mix", "penetration"], [meas_case, pen])
        vehicle_data = pd.concat([vehicle_data, vd_])

In [ ]:
vehicle_data = vehicle_data.reset_index()
part_type = []
for i,r in vehicle_data.iterrows():
    part_type.append("participant" if r.controlled else "non-participant")
    
vehicle_data["part_type"] = part_type

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

g = sns.catplot(
    data=vehicle_data, x="penetration", y="paid_price", hue="mix", col="part_type",
    capsize=.2, palette="Blues", errorbar=None,
    kind="point", height=4, aspect=1.2,
    marker = ["o", "^", "v"]
)

g.despine(left=True)
g.set_xticklabels([f"{i}%" for i in range(0, 120, 20)])
g.set_ylabels("actual paid price [€]")
g.set_titles("{col_name}")

In [ ]:
vehicle_data.groupby(["mix", "penetration", "controlled"])["paid_price"].mean()

In [ ]:
vehicle_data

In [ ]:
#price
import seaborn as sns
sns.reset_orig()

markers = ["o", "^", "v"]
colors = ["slategrey", "skyblue", "tab:blue"]

plt.figure(figsize=(4,4))

for i, meas_case in enumerate(MEAS_CASES):
    plot_points_y = []
    for pen in PENS:
        price = vehicle_data[np.logical_and(
            vehicle_data["mix"] == meas_case,
            vehicle_data["penetration"] == pen
        )]
        plot_points_y.append(price["auction_price"].mean())
    plt.plot(np.array(PENS)*10, plot_points_y, color=colors[i])
    plt.scatter(np.array(PENS)*10, plot_points_y, marker=markers[i],
                label=meas_case,
                color=colors[i])
    
plt.legend()
plt.xlabel("penetration [%]")
plt.ylabel("avg. auctioned price [€]")

In [ ]:
markers = ["o", "^", "v"]
colors = ["slategrey", "skyblue", "tab:blue"]

plt.figure(figsize=(4,4))

for i, meas_case in enumerate(MEAS_CASES):
    plot_points_y = []
    for pen in PENS:
        vd_filt = vehicle_data[
            np.logical_and(vehicle_data["mix"] == meas_case,
                           vehicle_data["penetration"] == pen)
        ]
        plot_points_y.append(vd_filt["occupied_reserved"].sum()/vd_filt["controlled"].sum())
    plt.plot(np.array(PENS)*10, plot_points_y, color=colors[i])
    plt.scatter(np.array(PENS)*10, plot_points_y, marker=markers[i],
                label=meas_case,
                color=colors[i])
    
plt.legend()
plt.xlabel("penetration [%]")
plt.ylabel("successful reservation rate")
#plt.title("auction successful reservation")

## Occupancy

In [ ]:
def combine_occups(meas_series, meas_name, seeds):
    meas_path = f"{MEAS_ROOT}/{meas_series}/{meas_name}"
    occupancies = pd.DataFrame()
    for seed in seeds:
        occup_df = pd.read_csv(f"{meas_path}/{seed}/occupancy.csv")
        occup_df["seed"] = seed
        occupancies = pd.concat([occupancies, occup_df])
    return occupancies

def plot_occupancy(occup_df):
    occup_filt = occup_df #no warning
    occup_filt["parking_id"] = occup_df["parking_id"].str.strip("pa")
    
    mean_occupancy = occup_filt.groupby("parking_id").mean()["occupancy"]
    occup_plot = {}
    for pid in mean_occupancy.index:
        occup_plot[pid] = mean_occupancy[pid]
    
    fig, ax = visualization.plot_network_probs(
        "../01_simulation/02_scenario/grid.net.xml",
        occup_plot,
        cmap="RdBu", #p_min = 0.0,  p_max=1.2
    )
    return fig, ax